In [ ]:
# !pip install faiss-cpu sentence-transformers langchain scikit-learn ragas pandas google-generativeai

In [51]:
import os
import google.generativeai as genai

In [52]:
genai.configure(api_key='API-key')
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)

인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등 다양한 인지 기능을 수행하는 컴퓨터 시스템 또는 기술입니다.



In [53]:
import faiss
import numpy as np
import pickle
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
import google.generativeai as genai

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision,context_recall

In [54]:
def create_faiss_index(texts, model, save_path="faiss_index.pkl"):
    """텍스트 리스트를 임베딩 후 FAISS에 저장 및 저장 파일 생성"""
    embeddings = model.encode(texts, convert_to_numpy=True)
    embeddings = normalize(embeddings, axis=1)  # 코사인 유사도 적용을 위해 정규화
    index = faiss.IndexFlatIP(embeddings.shape[1])  # 코사인 유사도 기반 Index
    index.add(embeddings)

    # FAISS 인덱스 및 텍스트 저장
    with open(save_path, "wb") as f:
        pickle.dump({"index": index, "texts": texts}, f)

    return index, embeddings

def load_faiss_index(load_path="faiss_index.pkl"):
    """저장된 FAISS 인덱스를 로드"""
    with open(load_path, "rb") as f:
        data = pickle.load(f)
    return data["index"], data["texts"]

def retrieve_documents(query, index, model, texts, top_k=5):
    """입력 질문과 가장 유사한 문서 청크 top_k 개 반환"""
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding, axis=1)  # 정규화
    distances, indices = index.search(query_embedding, top_k)
    retrieved_chunks = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return sorted(retrieved_chunks, key=lambda x: x[1], reverse=True)  # 유사도 순 정렬

def generate_gemini_answer(query, context):
    """Gemini API를 활용하여 답변 생성"""

    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content(query)
    return response.text


In [55]:
# 1️⃣ 텍스트 청킹 (줄넘김 기준, 500자 청크, 50자 오버랩)
file_path = "/Users/masang/Desktop/aiffel/mini-aiffelton/tistory_texts.txt"  # 사용할 .txt 파일 경로 지정
with open(file_path, "r", encoding="utf-8") as file:
    pages = file.readlines()

text_splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50, length_function=len)
texts = text_splitter.split_text("\n".join(pages))

for text in texts[:5]:  
    print(text)

Created a chunk of size 507, which is longer than the specified 500
Created a chunk of size 1013, which is longer than the specified 500
Created a chunk of size 716, which is longer than the specified 500
Created a chunk of size 656, which is longer than the specified 500
Created a chunk of size 779, which is longer than the specified 500


https://level.goorm.io/exam/195698/연합/quiz/1
백준 바이러스와 유사한 문제이다. 그래프로 연결할 때, 몇 개의 집합이 있는가 묻는 문제이다.
DFS와 BFS 방식 두 가지으로 모두 해결했지만, 여기에는 조금 다른 방법을 적어본다.
3주차에서 '탐색'이라는 카테고리에서 DFS와 BFS 응용을 했으니, 4주차에서는 진짜 '그래프'로 풀어보고자 했다.
구름의 출제 의도가 그것이 아닐까? 라는 생각이 들어 다른 그래프 알고리즘을 적용해보았다.
그게 아니라면 DFS와 BFS를 굳이 2주에 걸쳐 할 필요가 없으니 말이다.
Union-Find, 다른 말로는 DSU(Disjoint Set Union)으로 부르는 알고리즘이다.
각 노드(node)를 돌면서, 연결된 노드들 사이에서 root를 찾아 저장한다.
모든 노드를 돌면 각 노드의 저장값은, 루트 노드만이 존재하게 된다.
중복을 제거하면 루트 노드들의 값이 1개씩만 존재할 것이고, 이는 곧 집합의 수와 같다.
중복을 제거하면 루트 노드들의 값이 1개씩만 존재할 것이고, 이는 곧 집합의 수와 같다.
밑에서 설명할 알고리즘에서는 Union-Find라는 용어를 기준으로 진행한다.
Union-Find에서 Find를 담당하는 알고리즘이다.
탐색이라는 뜻의 Find로, 특정 노드가 어떤 루트 노드에 속한 그래프인지 탐색하는 알고리즘이다.
parent는 특정 index에 index의 루트 노드를 저장한 list다.
저장한 값과 index가 같다면, 노드 자신이 루트 노드다. 즉, 현재 값(current)를 반환하면 된다.
하지만 저장한 값과 index가 다르다면, 저장한 값을 따라서 루트 노드를 찾아간다.
Union-Find에서 Union을 담당하는 알고리즘이다.
조합이라는 뜻의 Union로, 각 노드들을 특정 루트 노드에 속하게 그래프를 합치는 알고리즘이다.
두 개의 start, end 노드를 입력받고, 두 노드를 루트 노드를 기준으로 하나의 그래프로 합친다.
밑의 함수에서 살펴보겠지만, Union(

In [56]:
# 2️⃣ KoSBERT 임베딩 생성 및 FAISS 저장
model = SentenceTransformer('jhgan/ko-sbert-sts')
index, embeddings = create_faiss_index(texts, model)

In [57]:
# 3️⃣ FAISS 인덱스 로드 (저장된 인덱스를 재사용 가능)
index, texts = load_faiss_index()

# 4️⃣ RAG 검색 실행 (질문 입력)
query = "역전파에 대해 설명해줘" # 질문 입력
retrieved_docs = retrieve_documents(query, index, model, texts, top_k=3)

# 5️⃣ 검색 결과 출력
for doc, score in retrieved_docs:
    print(f"유사도: {score:.4f} | 내용: {doc[:500]}...")

유사도: 0.4692 | 내용: 바로 이 과정에 사용하는 것이 바로 '푸리에 변환'이다.
조수 예측을 아주 제대로 사용한 것이 '제 2차 세계대전'이다.
연합군이 재빠른 침공을 위해 조수를 예측하여, 언제가 가장 유리한지 파악하게 해주었다.
제 2차 세계 대전을 하니 생각나는 이야기가 하나 더 있다.
제 2차 세계대전에서 미국은 나가사키와 히로시마에 원자폭탄을 투하했다.
이로 인해 전세계 사람들은 에너지의 위험성을 알게 되었다.
물론 그것은 방대한 에너지 때문만은 아닐 것이다. 강력한 방사능을 포함하고 있으니 문제가 되는 것이다.
그래서 각국에서 핵 연료에 대한 모든 활동을 금지하는 조항을 체결하기로 했다.
하지만 당시 소련은, 미국의 핵 군사력 강화를 위한 책략이라 여겼고, 곧 의미가 퇴색했다.
타국에서 핵을 연구하는지 아닌지 감시해야만 하는 상황에 닥쳤다.
방사성 동위원소가 대기 중에서는 수 천km를 날아가기 때문에 감지가 쉽다.
해저에서도 특수한 소리를 잡는 청음기로 포착이 쉽다....
유사도: 0.3727 | 내용: 해저에서도 특수한 소리를 잡는 청음기로 포착이 쉽다.
하지만 지하라면 이야기가 달라진다. 방사성 동위원소도 소리도 널리 퍼지지 않는다.
오직 진동만이 전달되는데, 이를 잡기 위해 사용한 것이 '푸리에 변환'이다.
그리고 이 계산을 빠르게 하기 위해서 '고속 푸리에 변환'이 나온 것이다.
빠르게 적국의 핵 실험을 포착해야 하기에 당연한 이유이다.
(참고로 위 영상도Veritasium의 유튜브이다.)
필요가 있어야 배움이 잘 된다.
FFT를 알기 위해서는 DFT를 알아야 한다.
DFT를 알기 위해서는 FT를 알아야 하고, Convolution을 알아야 한다.
하나하나 근원을 따라가며 차근차근 이해를 하면서 거슬러 올라왔다.
위 내용을 전부 이해했더라도, 코드로 옮기는 과정은 또 다른 일이었다.
그래도 많은 참고 자료를 확인하면서 AC를 뚫는 쾌거를 이뤘다.
위에서 설명한 수학 개념들과 유도 과정들은, 참고 자료와 함께 글로 작성할 예정이다...

In [58]:
# 7️⃣ Gemini API를 활용한 답변 생성
context_text = "\n".join([doc[0] for doc in retrieved_docs])
answer = generate_gemini_answer(query, context_text)
print("\n🔹 Gemini 응답:", answer)


🔹 Gemini 응답: ## 역전파(Backpropagation)란 무엇일까요?

역전파는 인공 신경망(Artificial Neural Network)을 훈련시키는 데 사용되는 핵심 알고리즘입니다. 신경망의 예측이 실제 값과 얼마나 다른지 (오차) 계산하고, 이 오차를 네트워크를 거슬러 올라가면서 각 가중치(weight)와 편향(bias)에 대한 기여도를 계산하여 가중치와 편향을 업데이트하는 방식으로 작동합니다.  즉, **신경망이 더 정확한 예측을 할 수 있도록 조정하는 과정**이라고 할 수 있습니다.

**좀 더 쉽게 비유를 들어 설명해 보겠습니다.**

*   **신경망:** 복잡한 미로
*   **가중치 및 편향:** 미로의 갈림길에서 어느 방향으로 가야 할지 결정하는 표지판
*   **입력:** 미로의 입구
*   **출력:** 미로의 출구
*   **오차:** 출구에 도착했지만, 원하는 출구가 아님
*   **역전파:** 잘못된 출구에서부터 미로를 거슬러 올라가면서 어떤 표지판(가중치 및 편향)이 잘못되었는지 파악하고, 표지판의 방향을 조금씩 수정하여 다음번에는 원하는 출구로 향하도록 하는 과정

**역전파의 핵심 단계:**

1.  **순전파 (Forward Propagation):**
    *   입력 데이터가 신경망의 입력층을 통해 들어옵니다.
    *   각 층을 거치면서 가중치와 편향에 의해 변환되고 활성화 함수를 통과합니다.
    *   최종적으로 출력층에서 예측값이 생성됩니다.

2.  **오차 계산 (Loss Calculation):**
    *   신경망의 예측값과 실제 정답 간의 오차를 계산합니다.  이때 오차 함수 (Loss function)를 사용합니다.  일반적으로 평균 제곱 오차 (Mean Squared Error, MSE)나 교차 엔트로피 오차 (Cross-Entropy Error) 등이 사용됩니다.

3.  **역전파 (Backward Propagation):**
    *   계산된 오차를 사용하여 출력층에서부

In [59]:
# 4️⃣ RAG 검색 실행 (질문 입력)
query = "MySQL로 데이터베이스 만드는 방법 알려줘" # 질문 입력
retrieved_docs = retrieve_documents(query, index, model, texts, top_k=3)

# 5️⃣ 검색 결과 출력
for doc, score in retrieved_docs:
    print(f"유사도: {score:.4f} | 내용: {doc[:200]}...")

# 7️⃣ Gemini API를 활용한 답변 생성
context_text = "\n".join([doc[0] for doc in retrieved_docs])
answer = generate_gemini_answer(query, context_text)
print("\n🔹 Gemini 응답:", answer)

유사도: 0.5822 | 내용: 데이터베이스 내부에 테이블을 생성하기 위해서는 데이터베이스에 접근해야 한다.
use_db_name에 접근할 데이터베이스 이름을 적으면 된다.
이때 MySQL Workbench 좌측창에서 더블 클릭으로 접근해도 상관없다.
현재 선택한 데이터베이스를 확인한다.
데이터베이스 내부에 존재하는 테이블 목록을 보여준다.
특정 테이블의 정보를 확인한다.
특정 테이블의 ...
유사도: 0.4967 | 내용: 현재는 'cat_id' 값이 기본키로 들어가있으며, name, breed, age가 column으로 있다.
MySQL이라는 DBMS를 사용하면 당연히 데이터베이스를 다룰 줄 알아야 한다.
위의 그림을 예시로 들면, 4개의 각기 다른 색에 해당하는 구역이 하나의 Database이다.
각 색상에 들어있는 다양한 크기의 사각형들이 Table이다.
이번 글에서 가...
유사도: 0.4058 | 내용: https://level.goorm.io/exam/195698/연합/quiz/1
백준 바이러스와 유사한 문제이다. 그래프로 연결할 때, 몇 개의 집합이 있는가 묻는 문제이다.
DFS와 BFS 방식 두 가지으로 모두 해결했지만, 여기에는 조금 다른 방법을 적어본다.
3주차에서 '탐색'이라는 카테고리에서 DFS와 BFS 응용을 했으니, 4주차에서는 진짜 '그래...

🔹 Gemini 응답: MySQL로 데이터베이스를 만드는 방법은 크게 두 가지로 나눌 수 있습니다.

**1. MySQL 클라이언트 (명령줄) 사용:**

*   MySQL 서버에 접속하여 SQL 명령어를 직접 입력하여 데이터베이스를 생성하는 방법입니다.
*   GUI 툴 없이도 가능하며, 자동화된 스크립트 작성에 용이합니다.

**2. GUI 툴 사용:**

*   MySQL Workbench, Dbeaver, Navicat 등의 GUI 툴을 사용하여 시각적으로 데이터베이스를 생성하고 관리하는 방법입니다.
*   SQL 명령어에 익숙하지 않은 사용자도 쉽게 데이터베이스를 생성할 수 있습니

In [ ]:
# evaluate RAG with metrics of RAGAS
data = {
    "user_input": [query],
    "response": [answer],
    "retrieved_contexts": [[context_text]],
    "reference": ["\n".join([  
        "1️⃣ MySQL 서버에 접속하기:",  
        "   ```sh",  
        "   mysql -u root -p",  
        "   ```",  
        "   - `root`는 기본 사용자명이며, 비밀번호 입력 후 접속.",  
        "",  
        "2️⃣ 데이터베이스 생성하기:",  
        "   ```sql",  
        "   CREATE DATABASE mydatabase;",  
        "   ```",  
        "   - `mydatabase`는 원하는 데이터베이스 이름으로 변경 가능.",  
        "",  
        "3️⃣ 데이터베이스 확인하기:",  
        "   ```sql",  
        "   SHOW DATABASES;",  
        "   ```",  
        "   - `mydatabase`가 목록에 있는지 확인!",  
        "",  
        "4️⃣ 사용할 데이터베이스 선택하기:",  
        "   ```sql",  
        "   USE mydatabase;",  
        "   ```",  
        "",  
        "5️⃣ 테이블 생성하기:",  
        "   ```sql",  
        "   CREATE TABLE users (",  
        "       id INT AUTO_INCREMENT PRIMARY KEY,",  
        "       name VARCHAR(50),",  
        "       email VARCHAR(100) UNIQUE,",  
        "       created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP",  
        "   );",  
        "   ```",  
        "",  
        "6️⃣ 테이블 목록 확인하기:",  
        "   ```sql",  
        "   SHOW TABLES;",  
        "   ```",  
        "",  
        "7️⃣ 데이터 삽입하기:",  
        "   ```sql",  
        "   INSERT INTO users (name, email) VALUES ('Alice', 'alice@example.com');",  
        "   ```",  
        "",  
        "8️⃣ 데이터 조회하기:",  
        "   ```sql",  
        "   SELECT * FROM users;",  
        "   ```",  
        "",  
        "✅ 이제 MySQL에서 데이터베이스와 테이블을 만들고 데이터를 추가할 수 있음!"  
    ])]
}


dataset = Dataset.from_dict(data)

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

results = evaluate(dataset, metrics)

df = results.to_pandas()
print(df)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

                 user_input  \
0  MySQL로 데이터베이스 만드는 방법 알려줘   

                                  retrieved_contexts  \
0  [데이터베이스 내부에 테이블을 생성하기 위해서는 데이터베이스에 접근해야 한다.\nu...   

                                            response  \
0  MySQL로 데이터베이스를 만드는 방법은 크게 두 가지로 나눌 수 있습니다.\n\n...   

                                           reference  faithfulness  \
0  1️⃣ MySQL 서버에 접속하기:\n   ```sh\n   mysql -u roo...           0.0   

   answer_relevancy  context_precision  context_recall  
0          0.857186                1.0             0.0  
